## 1 · Imports & Environment

In [25]:

from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent
from langchain.agents import AgentExecutor
from typing_extensions import Annotated, TypedDict
from typing import Callable, Literal, Optional, Sequence, Type, TypeVar, Union, cast, List, Tuple, Any
from langchain_core.messages import AIMessage, BaseMessage, SystemMessage, ToolMessage, HumanMessage
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph
from langgraph.prebuilt.tool_node import ToolNode, tools_condition
from langgraph.graph import MessagesState, START, END
from langgraph.types import Command
import pandas as pd
from pydantic import Field
import json
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.tools import tool


## 2 · Subagents & Utilities

In [26]:

from sql_react_agent_llama import SQL_SUBAGENT, make_dataframe
from viz_agent import VIZ_AGENT
from initiate_llm import gpt_llm, llama_llm


## 3 · AgentState TypedDict

In [27]:

class AgentState(TypedDict, total=False):
    """The shared state passed between nodes."""
    messages: Annotated[Sequence[BaseMessage], add_messages]
    question: Annotated[str, Field(description="The user question")]
    sql_query: Annotated[str, Field(description="Generated SQL query")]
    results: Annotated[List[List[Union[int, float, str, bool]]], Field(description="Raw SQL results")]
    df: Annotated[pd.DataFrame, Field(description="DataFrame built from results")]
    python_visualization_code: Annotated[str, Field(description="Code for visualization")]
    figures: Annotated[List[Any], Field(default_factory=list, description="Plotly Figure objects")]  # new
    pdf_bytes: Annotated[bytes, Field(default=b'', description="Generated PDF bytes")]               # new
    report_ready: Annotated[bool, Field(default=False, description="Report generation flag")]      # new


## 4 · Router Output Schema

In [28]:

class Router(TypedDict):
    """Next-step directive for supervisor node."""
    next: Literal["sql_agent", "make_table_node", "viz_agent", "report_agent", "FINISH"]
    final_answer: str


## 5 · SQL Generation & Execution Node

In [ ]:

import uuid
def nl2sql_node(state: AgentState) -> Command[Literal["supervisor"]]:
    # Invoke SQL subagent
    resp = SQL_SUBAGENT.invoke({
        "messages": state["messages"],
        "question": state["question"]
    })
    # sql = resp["query"]
    # results = resp["results"]
    # # Update state
    # state["sql_query"] = sql
    # state["results"] = results
    # state["messages"].append(ToolMessage(content=f"SQL executed: {sql}", tool_name="sql_agent"))
    # return Command(update={"sql_query": sql, "results": results}, goto="supervisor")
    
    print("SQL_SUBAGENT response:", resp)
    # Extract the last ToolMessage
    last_message = resp["messages"][-1]
    # Parse the JSON content of the ToolMessage
    data = json.loads(last_message.content)
    print("Parsed ToolMessage content:", data)
    # Get query and results
    sql = data.get("query", "")
    results = data.get("result", [])
    tool_call_id = str(uuid.uuid4())
    if not sql:
        state["messages"].append(ToolMessage(content="Failed to generate SQL query", tool_name="sql_agent", tool_call_id=tool_call_id))
    else:
        state["messages"].append(ToolMessage(content=f"SQL executed: {sql}", tool_name="sql_agent"))
    # Update state
    state["sql_query"] = sql
    state["results"] = results
    return Command(update={"sql_query": sql, "results": results}, goto="supervisor")


## 6 · DataFrame Construction Node

In [30]:

def make_table_node(state: AgentState) -> Command[Literal["supervisor"]]:
    df = make_dataframe(state["sql_query"], state["results"])
    state["df"] = df
    state["messages"].append(ToolMessage(content="DataFrame constructed", tool_name="make_table_node"))
    return Command(update={"df": df}, goto="supervisor")


## 7 · Visualization Node

In [31]:

def viz_node(state: AgentState) -> Command[Literal["supervisor"]]:
    # Use viz agent
    viz_input = {"question": state["question"], "df": state["df"], "results": state["results"]}
    viz_out = VIZ_AGENT.invoke(viz_input)
    # Store figure
    fig = viz_out["fig"]
    state.setdefault("figures", []).append(fig)
    state["python_visualization_code"] = viz_out["code"]
    state["messages"].append(ToolMessage(content="Visualization generated", tool_name="viz_agent"))
    return Command(update={"python_visualization_code": viz_out["code"]}, goto="supervisor")


## 8 · Report Generation Node

In [33]:

# Helper to build PDF
import io, datetime, textwrap
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image, PageBreak
import plotly.io as pio

def build_conversation_report(chat, figs, title="Dynamic Query-Viz Report"):
    buf = io.BytesIO()
    doc = SimpleDocTemplate(buf, pagesize=letter)
    styles = getSampleStyleSheet()
    story = [Paragraph(title, styles["Title"]), Spacer(1,12),
             Paragraph(datetime.datetime.now().strftime("Generated %B %d, %Y %I:%M %p"), styles["Normal"]),
             PageBreak()]
    # Transcript
    for m in chat:
        if isinstance(m, HumanMessage): role="User"
        elif isinstance(m, AIMessage): role="Assistant"
        else: continue
        story.append(Paragraph(f"<b>{role}:</b> {m.content}", styles["Normal"]))
        story.append(Spacer(1,8))
    # Figures
    if figs:
        story.append(PageBreak())
        for idx, fig in enumerate(figs,1):
            img_path = f"/tmp/plot_{idx}.png"
            fig.write_image(img_path, width=640, height=420, scale=2)
            story.extend([Paragraph(f"Figure {idx}", styles["Heading3"]),
                          Image(img_path, width=512, height=336),
                          Spacer(1,24)])
    doc.build(story)
    return buf.getvalue()

def report_node(state: AgentState) -> Command[Literal["supervisor"]]:
    pdf = build_conversation_report(state["messages"], state.get("figures", []))
    state["pdf_bytes"] = pdf
    state["report_ready"] = True
    state["messages"].append(AIMessage(content="Report generated: report.pdf"))
    return Command(update={"pdf_bytes": pdf, "report_ready": True}, goto="supervisor")


#### Setting up Supervisor

In [34]:
# Define available agents
members = ["sql_agent", "make_table", "viz_agent"]
# Add FINISH as an option for task completion
options = members + ["FINISH"]


## 9 · Supervisor Prompt

In [35]:

supervisor_prompt = f"""
You are a supervisor agent named "SQLOrchestrator" tasked with managing the following workers: {{members}}.

1. SQL Generation & Execution:
   - If results are empty, call `sql_agent`.

2. Table Construction:
   - If results exist but no `df`, call `make_table_node`.

3. Visualization:
   - If `df` exists but no `figures`, call `viz_agent`.

4. Report Generation:
   - If the user asks "generate report" and `report_ready` is False, call `report_agent`.

When everything needed is complete (results, df, viz), and after report if requested, output FINISH with a final answer.
""".format(members=members)


## 10 · Supervisor Node

In [36]:

def supervisor_node(state: AgentState) -> Command:
    messages = state["messages"]
    # Add routing instruction
    response = gpt_llm.with_structured_output(Router).invoke(messages)
    goto = response["next"]
    if goto == "report_agent":
        return Command(goto="report_agent")
    if goto == "FINISH":
        # send final answer
        state["messages"].append(HumanMessage(content=response["final_answer"], name="supervisor"))
        return Command(goto=END, update={"messages": state["messages"]})
    return Command(goto=goto)


## 11 · Build & Compile Graph

In [37]:
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage

builder = StateGraph(AgentState)

# Register every node
builder.add_node("supervisor", supervisor_node)
builder.add_node("sql_agent", nl2sql_node)
builder.add_node("make_table_node", make_table_node)
builder.add_node("viz_agent", viz_node)
builder.add_node("report_agent", report_node)

# Start → supervisor
builder.add_edge(START, "supervisor")

# Supervisor conditional routing
builder.add_conditional_edges(
    "supervisor",
    lambda s: (
        "sql_agent" if not s.get("results")
        else "make_table_node" if s.get("results") and not s.get("df")
        else "viz_agent" if s.get("df") and not s.get("figures")
        else "report_agent" if any(
            isinstance(m, HumanMessage) and "generate report" in m.content.lower()
            for m in s.get("messages", [])
        ) and not s.get("report_ready", False)
        else END if (
            s.get("df") is not None
            and s.get("figures")
            and (
                s.get("report_ready", False)
                or not any(
                    isinstance(m, HumanMessage)
                    and "generate report" in m.content.lower()
                    for m in s.get("messages", [])
                )
            )
        ) else None
    ),
    {
        "sql_agent": "sql_agent",
        "make_table_node": "make_table_node",
        "viz_agent": "viz_agent",
        "report_agent": "report_agent",
        END: END
    }
)

# Loop every worker back to supervisor
builder.add_edge("sql_agent", "supervisor")
builder.add_edge("make_table_node", "supervisor")
builder.add_edge("viz_agent", "supervisor")
builder.add_edge("report_agent", "supervisor")

# Compile the graph
graph = builder.compile()

## 12 · Demo Invocation

In [38]:
from langchain_core.messages import HumanMessage

state = {
    "messages": [
        HumanMessage(content="show me ethnicity of all the employees"),
        HumanMessage(content="Generate report")
    ],
    "question": "Show me ethnicity of all the employees"
}

final_state = graph.invoke(state)

with open("report.pdf", "wb") as f:
    f.write(final_state["pdf_bytes"])

print("report.pdf saved, size:", len(final_state["pdf_bytes"]))




=======IN SQL TOOL WITH ==> The report detailing the ethnicity of all employees is as follows:

## **Employee Ethnicity Report**

| Employee ID | Employee Name     | Ethnicity            |
|-------------|------------------|----------------------|
| 001         | John Smith       | Caucasian            |
| 002         | Maria Garcia     | Hispanic             |
| 003         | Chen Wei         | Asian                |
| 004         | Amina Mohammed   | Middle Eastern       |
| 005         | David Johnson    | African American      |
| 006         | Priya Patel      | Asian                |
| 007         | Nia Johnson      | African American      |
| 008         | Carlos Ramirez   | Hispanic             |
| 009         | Sophie Dubois    | Caucasian            |
| 010         | Omar Khayyam     | Middle Eastern       |

### **Summary:**
- **Caucasian:** 3
- **Hispanic:** 3
- **Asian:** 2
- **Middle Eastern:** 2
- **African American:** 2

This report can be used for diversity analysis a

KeyError: 'tool_call_id'